# Ejemplos Prácticos de Machine Learning
## Conceptos Fundamentales con Código Python

Este notebook contiene ejemplos prácticos de diferentes técnicas de Machine Learning aplicadas a problemas reales diversos.

### Contenido:
1. Regresión Lineal - Predicción de precios de casas
2. Clasificación - Detección de spam
3. Clustering - Segmentación de clientes
4. Series Temporales - Predicción de demanda
5. NLP - Análisis de sentimientos

## Configuración Inicial

In [ ]:
# Importar librerías esenciales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
%matplotlib inline

# Semilla para reproducibilidad
np.random.seed(42)

print("✅ Librerías cargadas correctamente")

---
## 1. REGRESIÓN LINEAL: Predicción de Precios de Casas

**Objetivo**: Predecir el precio de una casa basándose en sus características

**Tipo**: Regresión (valor continuo)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression

# Generar datos sintéticos de casas
print("🏠 PREDICCIÓN DE PRECIOS DE CASAS\n")

# Crear dataset sintético
np.random.seed(42)
n_samples = 500

# Features: tamaño (m²), habitaciones, antigüedad, distancia centro
tamano = np.random.uniform(50, 300, n_samples)  # m²
habitaciones = np.random.randint(1, 6, n_samples)
antiguedad = np.random.uniform(0, 50, n_samples)  # años
distancia_centro = np.random.uniform(0, 30, n_samples)  # km

# Precio basado en fórmula con ruido
precio = (
    2000 * tamano +  # €2000 por m²
    15000 * habitaciones +  # €15000 por habitación
    -500 * antiguedad +  # Depreciación por año
    -1000 * distancia_centro +  # Menos valor lejos del centro
    np.random.normal(0, 20000, n_samples)  # Ruido
)

# Crear DataFrame
df_casas = pd.DataFrame({
    'tamano_m2': tamano,
    'habitaciones': habitaciones,
    'antiguedad_anos': antiguedad,
    'distancia_centro_km': distancia_centro,
    'precio': precio
})

print("Dataset creado con", len(df_casas), "casas")
print("\nPrimeras filas:")
display(df_casas.head(10))

print("\nEstadísticas descriptivas:")
display(df_casas.describe())

In [ ]:
# Exploración visual
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Scatter plots de cada feature vs precio
features = ['tamano_m2', 'habitaciones', 'antiguedad_anos', 'distancia_centro_km']
titles = ['Tamaño vs Precio', 'Habitaciones vs Precio', 'Antigüedad vs Precio', 'Distancia vs Precio']

for idx, (feature, title) in enumerate(zip(features, titles)):
    ax = axes[idx // 2, idx % 2]
    ax.scatter(df_casas[feature], df_casas['precio'], alpha=0.5)
    ax.set_xlabel(feature)
    ax.set_ylabel('Precio (€)')
    ax.set_title(title)
    
    # Línea de tendencia
    z = np.polyfit(df_casas[feature], df_casas['precio'], 1)
    p = np.poly1d(z)
    ax.plot(df_casas[feature], p(df_casas[feature]), "r--", alpha=0.8, linewidth=2)

plt.tight_layout()
plt.show()

In [ ]:
# Preparar datos para el modelo
X = df_casas[features]
y = df_casas['precio']

# Dividir en train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Conjunto de entrenamiento: {len(X_train)} casas")
print(f"Conjunto de prueba: {len(X_test)} casas")

# Entrenar modelo de regresión lineal
modelo_precio = LinearRegression()
modelo_precio.fit(X_train, y_train)

print("\n✅ Modelo entrenado")
print("\nCoeficientes aprendidos:")
for feature, coef in zip(features, modelo_precio.coef_):
    print(f"  {feature:25s}: {coef:>10,.2f} €")
print(f"  {'Intercepto':25s}: {modelo_precio.intercept_:>10,.2f} €")

In [ ]:
# Evaluar el modelo
y_pred_train = modelo_precio.predict(X_train)
y_pred_test = modelo_precio.predict(X_test)

# Métricas
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print("📊 MÉTRICAS DEL MODELO:\n")
print(f"R² (Train): {r2_train:.3f}")
print(f"R² (Test):  {r2_test:.3f}")
print(f"\nRMSE (Train): €{rmse_train:,.2f}")
print(f"RMSE (Test):  €{rmse_test:,.2f}")

print("\n💡 Interpretación:")
print(f"  - El modelo explica el {r2_test*100:.1f}% de la varianza en los precios")
print(f"  - Error promedio de predicción: €{rmse_test:,.0f}")

In [ ]:
# Visualización de predicciones vs realidad
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot predicho vs real
ax1 = axes[0]
ax1.scatter(y_test, y_pred_test, alpha=0.6)
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Predicción perfecta')
ax1.set_xlabel('Precio Real (€)')
ax1.set_ylabel('Precio Predicho (€)')
ax1.set_title(f'Predicciones vs Realidad (R²={r2_test:.3f})')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Distribución de errores
ax2 = axes[1]
errores = y_test - y_pred_test
ax2.hist(errores, bins=30, edgecolor='black', alpha=0.7)
ax2.axvline(x=0, color='r', linestyle='--', linewidth=2, label='Error = 0')
ax2.set_xlabel('Error de Predicción (€)')
ax2.set_ylabel('Frecuencia')
ax2.set_title('Distribución de Errores')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Ejemplo de predicción para una casa nueva
print("🏡 PREDICCIÓN PARA CASA NUEVA:\n")

casa_nueva = pd.DataFrame({
    'tamano_m2': [120],
    'habitaciones': [3],
    'antiguedad_anos': [10],
    'distancia_centro_km': [5]
})

precio_predicho = modelo_precio.predict(casa_nueva)[0]

print("Características de la casa:")
print(f"  - Tamaño: {casa_nueva['tamano_m2'][0]} m²")
print(f"  - Habitaciones: {casa_nueva['habitaciones'][0]}")
print(f"  - Antigüedad: {casa_nueva['antiguedad_anos'][0]} años")
print(f"  - Distancia al centro: {casa_nueva['distancia_centro_km'][0]} km")
print(f"\n💰 Precio predicho: €{precio_predicho:,.2f}")

# Intervalo de confianza aproximado
intervalo = 1.96 * rmse_test  # 95% confianza
print(f"\nIntervalo de confianza (95%):")
print(f"  Entre €{precio_predicho - intervalo:,.2f} y €{precio_predicho + intervalo:,.2f}")

---
## 2. CLASIFICACIÓN: Detección de Spam en Emails

**Objetivo**: Clasificar emails como spam o no-spam

**Tipo**: Clasificación binaria

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix

print("📧 DETECCIÓN DE SPAM EN EMAILS\n")

# Crear dataset sintético de emails
emails_spam = [
    "OFERTA INCREÍBLE!!! Compra ahora y ahorra 90%",
    "¡¡¡GANA DINERO RÁPIDO!!! Haz clic aquí",
    "Enhorabuena has ganado un iPhone gratis",
    "Oferta exclusiva solo para ti !!! URGENTE",
    "Descuento del 99% en productos de lujo",
    "Haz click aquí para ganar 1000 euros",
    "VIAGRA barata compra ahora sin receta",
    "Trabajo desde casa gana dinero fácil",
    "Promoción limitada oferta increíble descuento",
    "Premio lotería has ganado millones"
] * 5  # Repetir para tener más datos

emails_legit = [
    "Reunión el lunes a las 10:00 en la sala de juntas",
    "Hola, adjunto el informe que pediste",
    "Confirmación de tu pedido número 12345",
    "Gracias por tu compra, aquí está tu factura",
    "Recordatorio: cita médica mañana a las 3pm",
    "El proyecto está avanzando según lo planeado",
    "Feliz cumpleaños, espero que pases un gran día",
    "Te envío las fotos de las vacaciones",
    "Necesito que revises este documento por favor",
    "La reunión de mañana se ha cancelado"
] * 5

# Crear DataFrame
df_emails = pd.DataFrame({
    'texto': emails_spam + emails_legit,
    'es_spam': [1]*len(emails_spam) + [0]*len(emails_legit)
})

# Mezclar
df_emails = df_emails.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Dataset: {len(df_emails)} emails")
print(f"  - Spam: {df_emails['es_spam'].sum()}")
print(f"  - Legítimo: {len(df_emails) - df_emails['es_spam'].sum()}")
print("\nEjemplos:")
display(df_emails.sample(10))

In [ ]:
# Vectorización de texto con TF-IDF
vectorizer = TfidfVectorizer(max_features=100)
X_emails = vectorizer.fit_transform(df_emails['texto'])
y_emails = df_emails['es_spam']

print("Vocabulario aprendido (primeras 20 palabras):")
print(vectorizer.get_feature_names_out()[:20])
print(f"\nDimensionalidad: {X_emails.shape}")
print(f"  - {X_emails.shape[0]} emails")
print(f"  - {X_emails.shape[1]} features (palabras)")

In [ ]:
# Dividir en train/test
X_train_email, X_test_email, y_train_email, y_test_email = train_test_split(
    X_emails, y_emails, test_size=0.3, random_state=42, stratify=y_emails
)

# Entrenar clasificador Naive Bayes
clf_spam = MultinomialNB()
clf_spam.fit(X_train_email, y_train_email)

# Predicciones
y_pred_email = clf_spam.predict(X_test_email)

# Evaluación
accuracy = accuracy_score(y_test_email, y_pred_email)

print("✅ Modelo entrenado")
print(f"\nAccuracy: {accuracy:.2%}")
print("\nReporte de clasificación:")
print(classification_report(y_test_email, y_pred_email, 
                           target_names=['Legítimo', 'Spam']))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test_email, y_pred_email)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=['Legítimo', 'Spam'],
           yticklabels=['Legítimo', 'Spam'])
plt.ylabel('Real')
plt.xlabel('Predicho')
plt.title('Matriz de Confusión - Detector de Spam')

# Añadir explicación
tn, fp, fn, tp = cm.ravel()
plt.text(1.5, 2.5, f'VP={tp}\nFP={fp}\nFN={fn}\nVN={tn}', 
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("\n💡 Interpretación:")
print(f"  - Verdaderos Positivos (TP): {tp} - Spam correctamente detectado")
print(f"  - Falsos Positivos (FP): {fp} - Legítimo marcado como spam (¡malo!)")
print(f"  - Falsos Negativos (FN): {fn} - Spam no detectado (¡malo!)")
print(f"  - Verdaderos Negativos (TN): {tn} - Legítimo correctamente identificado")

In [ ]:
# Probar con emails nuevos
print("🧪 PRUEBAS CON EMAILS NUEVOS:\n")

emails_prueba = [
    "Reunión mañana para discutir el proyecto",
    "¡¡¡OFERTA LIMITADA!!! Gana dinero rápido",
    "Confirmación de tu reserva de hotel",
    "COMPRA AHORA y ahorra 95% URGENTE"
]

X_prueba = vectorizer.transform(emails_prueba)
predicciones = clf_spam.predict(X_prueba)
probabilidades = clf_spam.predict_proba(X_prueba)

for email, pred, proba in zip(emails_prueba, predicciones, probabilidades):
    label = "📩 SPAM" if pred == 1 else "✅ LEGÍTIMO"
    confianza = proba[pred] * 100
    print(f"{label} ({confianza:.1f}% confianza)")
    print(f"  Email: '{email}'")
    print()

---
## 3. CLUSTERING: Segmentación de Clientes

**Objetivo**: Agrupar clientes en segmentos similares sin etiquetas previas

**Tipo**: Aprendizaje no supervisado

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

print("👥 SEGMENTACIÓN DE CLIENTES\n")

# Generar datos sintéticos de clientes
np.random.seed(42)
n_clientes = 300

# Crear 3 grupos naturales de clientes
# Grupo 1: Jóvenes, bajos ingresos, muchas compras pequeñas
grupo1 = pd.DataFrame({
    'edad': np.random.normal(25, 3, 100),
    'ingreso_anual': np.random.normal(25000, 5000, 100),
    'gasto_mensual': np.random.normal(200, 50, 100),
    'frecuencia_compra': np.random.normal(15, 3, 100)
})

# Grupo 2: Adultos, ingresos medios, compras moderadas
grupo2 = pd.DataFrame({
    'edad': np.random.normal(40, 5, 100),
    'ingreso_anual': np.random.normal(50000, 10000, 100),
    'gasto_mensual': np.random.normal(500, 100, 100),
    'frecuencia_compra': np.random.normal(8, 2, 100)
})

# Grupo 3: Seniors, altos ingresos, compras caras pero pocas
grupo3 = pd.DataFrame({
    'edad': np.random.normal(55, 7, 100),
    'ingreso_anual': np.random.normal(80000, 15000, 100),
    'gasto_mensual': np.random.normal(1000, 200, 100),
    'frecuencia_compra': np.random.normal(4, 1, 100)
})

df_clientes = pd.concat([grupo1, grupo2, grupo3], ignore_index=True)

print(f"Dataset: {len(df_clientes)} clientes")
print("\nPrimeras filas:")
display(df_clientes.head(10))

print("\nEstadísticas:")
display(df_clientes.describe())

In [ ]:
# Visualización inicial
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Edad vs Ingreso
axes[0].scatter(df_clientes['edad'], df_clientes['ingreso_anual'], alpha=0.6)
axes[0].set_xlabel('Edad')
axes[0].set_ylabel('Ingreso Anual (€)')
axes[0].set_title('Edad vs Ingreso')
axes[0].grid(True, alpha=0.3)

# Frecuencia vs Gasto
axes[1].scatter(df_clientes['frecuencia_compra'], df_clientes['gasto_mensual'], alpha=0.6)
axes[1].set_xlabel('Frecuencia de Compra (veces/mes)')
axes[1].set_ylabel('Gasto Mensual (€)')
axes[1].set_title('Frecuencia vs Gasto')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Se pueden observar posibles agrupaciones naturales en los datos")

In [ ]:
# Normalizar datos (importante para K-Means)
scaler = StandardScaler()
X_clientes_scaled = scaler.fit_transform(df_clientes)

# Método del codo para encontrar K óptimo
inertias = []
K_range = range(1, 10)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_clientes_scaled)
    inertias.append(kmeans.inertia_)

# Visualizar método del codo
plt.figure(figsize=(10, 5))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Número de Clusters (K)')
plt.ylabel('Inercia (Within-Cluster Sum of Squares)')
plt.title('Método del Codo para encontrar K óptimo')
plt.axvline(x=3, color='r', linestyle='--', label='K=3 (codo)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print("💡 El 'codo' sugiere que K=3 es óptimo")

In [ ]:
# Aplicar K-Means con K=3
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df_clientes['cluster'] = kmeans.fit_predict(X_clientes_scaled)

print("✅ Clustering completado")
print(f"\nDistribución de clientes por cluster:")
print(df_clientes['cluster'].value_counts().sort_index())

# Analizar características de cada cluster
print("\n📊 PERFIL DE CADA CLUSTER:\n")
cluster_profiles = df_clientes.groupby('cluster').mean()
display(cluster_profiles.round(2))

In [ ]:
# Visualización de clusters
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
colors = ['red', 'green', 'blue']

# Edad vs Ingreso coloreado por cluster
for cluster in [0, 1, 2]:
    mask = df_clientes['cluster'] == cluster
    axes[0].scatter(df_clientes.loc[mask, 'edad'], 
                   df_clientes.loc[mask, 'ingreso_anual'],
                   c=colors[cluster], label=f'Cluster {cluster}', alpha=0.6, s=50)
axes[0].set_xlabel('Edad')
axes[0].set_ylabel('Ingreso Anual (€)')
axes[0].set_title('Clusters: Edad vs Ingreso')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Frecuencia vs Gasto coloreado por cluster
for cluster in [0, 1, 2]:
    mask = df_clientes['cluster'] == cluster
    axes[1].scatter(df_clientes.loc[mask, 'frecuencia_compra'], 
                   df_clientes.loc[mask, 'gasto_mensual'],
                   c=colors[cluster], label=f'Cluster {cluster}', alpha=0.6, s=50)
axes[1].set_xlabel('Frecuencia de Compra (veces/mes)')
axes[1].set_ylabel('Gasto Mensual (€)')
axes[1].set_title('Clusters: Frecuencia vs Gasto')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Interpretación de clusters
print("🎯 INTERPRETACIÓN DE CLUSTERS:\n")

for cluster in [0, 1, 2]:
    perfil = cluster_profiles.loc[cluster]
    print(f"Cluster {cluster}:")
    print(f"  - Edad promedio: {perfil['edad']:.0f} años")
    print(f"  - Ingreso anual: €{perfil['ingreso_anual']:,.0f}")
    print(f"  - Gasto mensual: €{perfil['gasto_mensual']:,.0f}")
    print(f"  - Frecuencia: {perfil['frecuencia_compra']:.1f} compras/mes")
    
    # Etiqueta descriptiva
    if perfil['edad'] < 30:
        etiqueta = "🎓 Jóvenes Activos"
    elif perfil['edad'] < 50:
        etiqueta = "👔 Profesionales Establecidos"
    else:
        etiqueta = "💎 Premium Selectivos"
    
    print(f"  Etiqueta: {etiqueta}")
    print()

---
## Conclusiones

### Conceptos clave aprendidos:

1. **Regresión Lineal**:
   - Predice valores continuos
   - Interpreta coeficientes como impacto de cada feature
   - Métricas: R², RMSE, MAE

2. **Clasificación (Naive Bayes)**:
   - Predice categorías discretas
   - Matriz de confusión para analizar errores
   - Trade-off entre Precision y Recall

3. **Clustering (K-Means)**:
   - Encuentra grupos naturales sin etiquetas
   - Método del codo para elegir K
   - Normalización crucial para distancias

### Próximos pasos:

- Experimentar con datasets reales (Kaggle, UCI)
- Probar otros algoritmos (Random Forest, XGBoost, SVM)
- Aprender técnicas de feature engineering
- Estudiar validación cruzada y grid search
- Explorar deep learning para problemas complejos